In [143]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
import category_encoders as ce

In [144]:
train_data = pd.read_csv("./data/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [145]:
# removing useless columns
train_data = train_data.drop(columns=['PassengerId', 'Name'])
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [146]:
# Replace NaN in Age
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())

In [147]:
# Replace NaN in Cabin with Missing
train_data['Cabin'] = train_data['Cabin'].fillna('Missing')

In [148]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,Missing,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,Missing,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,Missing,S


## Encoder

In [149]:
train_data_X = train_data.iloc[:, 1:]

In [150]:
#onehot encoder for sex and embarked column
train_data_X = pd.concat([
    train_data_X, 
    pd.get_dummies(train_data_X['Sex'], prefix='Sex', drop_first=True),
    pd.get_dummies(train_data_X['Embarked'], prefix='Embarked', drop_first=True)
], axis=1)

# Drop the original 'Sex' and 'Embarked' columns as they are now encoded
train_data_X.drop(['Sex', 'Embarked'], axis=1, inplace=True)

# Show the result
train_data_X.head()

,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,A/5 21171,7.2500,Missing,True,False,True
1,1,38.0,1,0,PC 17599,71.2833,C85,False,False,False
2,3,26.0,0,0,STON/O2. 3101282,7.9250,Missing,False,False,True
3,1,35.0,1,0,113803,53.1000,C123,False,False,True
4,3,35.0,0,0,373450,8.0500,Missing,True,False,True


In [151]:
# BaseNencoding for Ticket and Cabin
base_n_encoder_cols = ['Ticket', 'Cabin']

# Initialize BaseNEncoder 
encoder = ce.BaseNEncoder(cols=base_n_encoder_cols, base=5)

# Fit the encoder on the data
train_data_X_encoded = encoder.fit_transform(train_data_X)

# Show the encoded DataFrame
train_data_X_encoded.head()


/opt/anaconda3/envs/Titanic_env/lib/python3.12/site-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(


,Pclass,Age,SibSp,Parch,Ticket_0,Ticket_1,Ticket_2,Ticket_3,Ticket_4,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,0,0,0,0,1,7.2500,0,0,0,1,True,False,True
1,1,38.0,1,0,0,0,0,0,2,71.2833,0,0,0,2,False,False,False
2,3,26.0,0,0,0,0,0,0,3,7.9250,0,0,0,1,False,False,True
3,1,35.0,1,0,0,0,0,0,4,53.1000,0,0,0,3,False,False,True
4,3,35.0,0,0,0,0,0,1,0,8.0500,0,0,0,1,True,False,True


## train validation split

In [152]:
X = train_data_X_encoded
y = train_data.iloc[:, 0]

In [153]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=42)

In [154]:
X_train.head()

,Pclass,Age,SibSp,Parch,Ticket_0,Ticket_1,Ticket_2,Ticket_3,Ticket_4,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Sex_male,Embarked_Q,Embarked_S
331,1,45.5,0,0,0,2,1,3,2,28.5000,0,2,3,0,True,False,True
733,2,23.0,0,0,0,4,3,1,2,13.0000,0,0,0,1,True,False,True
382,3,32.0,0,0,0,2,3,1,0,7.9250,0,0,0,1,True,False,True
704,3,26.0,1,0,0,4,2,2,2,7.8542,0,0,0,1,True,False,True
813,3,6.0,4,2,0,0,0,2,4,31.2750,0,0,0,1,False,False,True


## Model

In [155]:
# fit the model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=80, max_depth=4, random_state=1)
model.fit(X_train, y_train)


RandomForestClassifier(max_depth=4, n_estimators=80, random_state=1)

In [156]:
#make predictions on training set
y_train_pred = model.predict(X_train)

In [157]:
#make predictions on validation set
y_valid_pred = model.predict(X_valid)

In [158]:
from sklearn.metrics import accuracy_score, classification_report

In [159]:
# Evaluate the model
accuracy = accuracy_score(y_valid, y_valid_pred)
print("Accuracy on the validation set:", accuracy)


Accuracy on the validation set: 0.8100558659217877


In [160]:
# Evaluate the model
accuracy = accuracy_score(y_train, y_train_pred)
print("Accuracy on the training set:", accuracy)

Accuracy on the training set: 0.8623595505617978


In [161]:
print("Classification Report: validation set")
print(classification_report(y_valid, y_valid_pred))

Classification Report: validation set
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       105
           1       0.81      0.70      0.75        74

    accuracy                           0.81       179
   macro avg       0.81      0.79      0.80       179
weighted avg       0.81      0.81      0.81       179



In [162]:
print("Classification Report: training set")
print(classification_report(y_train, y_train_pred))

Classification Report: training set
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       444
           1       0.92      0.69      0.79       268

    accuracy                           0.86       712
   macro avg       0.88      0.83      0.84       712
weighted avg       0.87      0.86      0.86       712



## Preprocessing for test data

In [163]:
test_data = pd.read_csv("./data/test.csv")

In [164]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [165]:
# removing useless columns
test_data = test_data.drop(columns=['PassengerId', 'Name'])
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,3,female,47.0,1,0,363272,7.0000,NaN,S
2,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,3,male,27.0,0,0,315154,8.6625,NaN,S
4,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [166]:
test_data.isnull().sum()

Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [167]:
# Replace NaN in Age
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())

In [168]:
# Replace NaN in Cabin with Missing
test_data['Cabin'] = test_data['Cabin'].fillna('Missing')

In [169]:
#onehot encoder for sex and embarked column
test_data = pd.concat([
    test_data, 
    pd.get_dummies(test_data['Sex'], prefix='Sex', drop_first=True),
    pd.get_dummies(test_data['Embarked'], prefix='Embarked', drop_first=True)
], axis=1)

# Drop the original 'Sex' and 'Embarked' columns as they are now encoded
test_data.drop(['Sex', 'Embarked'], axis=1, inplace=True)

# Show the result
test_data.head()

,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,330911,7.8292,Missing,True,True,False
1,3,47.0,1,0,363272,7.0000,Missing,False,False,True
2,2,62.0,0,0,240276,9.6875,Missing,True,True,False
3,3,27.0,0,0,315154,8.6625,Missing,True,False,True
4,3,22.0,1,1,3101298,12.2875,Missing,False,False,True


In [170]:
# transform only the test data
test_data_encoded = encoder.transform(test_data)

# Show the encoded DataFrame
test_data_encoded.head()

,Pclass,Age,SibSp,Parch,Ticket_0,Ticket_1,Ticket_2,Ticket_3,Ticket_4,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Sex_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,0,0,0,0,0,7.8292,0,0,0,1,True,True,False
1,3,47.0,1,0,0,0,0,0,0,7.0000,0,0,0,1,False,False,True
2,2,62.0,0,0,0,0,0,0,0,9.6875,0,0,0,1,True,True,False
3,3,27.0,0,0,0,0,0,0,0,8.6625,0,0,0,1,True,False,True
4,3,22.0,1,1,0,3,1,1,1,12.2875,0,0,0,1,False,False,True


## prediction and submission

In [171]:
X_test = test_data_encoded

In [172]:
original_test_data = pd.read_csv("./data/test.csv")

In [173]:
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': original_test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
